#### Initialize universe and number of sets  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def create_instance(universe_size, num_sets, density):
    rng = np.random.default_rng()
    SETS = rng.random((num_sets, universe_size)) < density
    for s in range(universe_size):
        if not np.any(SETS[:, s]):
            SETS[rng.integers(num_sets), s] = True
    COSTS = np.pow(SETS.sum(axis=1), 1.1) 
    return SETS, COSTS

#### Return the number of covered elements for the solution

In [ ]:
def valid(solution, SETS):
    phenotype = np.logical_or.reduce(SETS[solution])
    covered_elements = np.sum(phenotype)
    return covered_elements

#### Find the cost of the solution

In [ ]:
def cost(solution, COSTS):
    return COSTS[solution].sum()

#### Two different tweak functions, first one will be used by hill climbing and the second one by simulated annealing

In [ ]:

def tweak_hill_climbing(solution, num_sets, rng, change_rate):
    new_sol = solution.copy()
    mask = rng.random(num_sets) < change_rate
    new_sol[mask] = ~new_sol[mask] 
    return new_sol

def tweak_simulated_annealing(solution, num_sets, rng):
    new_sol = solution.copy()
    i = rng.integers(num_sets)
    new_sol[i] = not solution[i] 
    return new_sol


#### Compute the fitness wich is the opposite of cost but adding a penalty for non covered items, the penalty change depending on the instance of the problem

In [ ]:
def fitness(solution, SETS, COSTS, universe_size, instance_scale):
    num_elements_covered = valid(solution, SETS)
    uncovered_elements = universe_size - num_elements_covered
    total_cost = cost(solution, COSTS)
    alpha = instance_scale
    return - (alpha * uncovered_elements + total_cost)

##### Hill climbing, the tweak solution changes the solution more in the initials steps and less then to favor exploration in the first phase and exploitation later


In [ ]:
# Hill Climbing
def hill_climbing(SETS, COSTS, universe_size, num_sets, steps=10000):
    rng = np.random.default_rng()
    solution = rng.random(num_sets) < 0.1  
    best_fitness = fitness(solution, SETS, COSTS, universe_size, num_sets)
    fitness_history = []  

    for step in range(steps):
        change_rate = 0.1 * (1 - step / steps)
        new_solution = tweak_hill_climbing(solution, num_sets, rng, change_rate)
        new_fitness = fitness(new_solution, SETS, COSTS, universe_size, num_sets)

        if new_fitness > best_fitness:
            solution = new_solution
            best_fitness = new_fitness

        fitness_history.append(best_fitness)

    return solution, best_fitness, fitness_history


#### Hill climbing seems to performs better when the dimensions of the problem are relatively small

In [ ]:
# Simulated Annealing
def simulated_annealing(SETS, COSTS, universe_size, num_sets, steps=10000, temp=2000.0, cooling_rate=0.95):

    rng = np.random.default_rng()
    solution = rng.random(num_sets) < 0.1  
    best_solution = solution
    best_fitness = fitness(solution, SETS, COSTS, universe_size, num_sets)
    fitness_history = []  

    current_temp = temp

    for step in range(steps):
        new_solution = tweak_simulated_annealing(solution, num_sets, rng)
        new_fitness = fitness(new_solution, SETS, COSTS, universe_size, num_sets)

        if new_fitness > best_fitness:
            solution = new_solution
            best_fitness = new_fitness
            best_solution = solution
        else:
            accept_prob = np.exp((new_fitness - best_fitness) / current_temp)
            if rng.random() < accept_prob:
                solution = new_solution
                best_fitness = new_fitness

        fitness_history.append(best_fitness)
        current_temp *= cooling_rate

    return best_solution, best_fitness, fitness_history

#### Choose wich algorithm use and do the computation

In [ ]:
def solve_scp_instance(universe_size, num_sets, density):
    SETS, COSTS = create_instance(universe_size, num_sets, density)
    
    if num_sets <= 100:
        print(f"Solving instance with {num_sets} sets using Hill Climbing.")
        best_solution, best_fitness, fitness_history = hill_climbing(SETS, COSTS, universe_size, num_sets)
    else:
        print(f"Solving instance with {num_sets} sets using Simulated Annealing.")
        best_solution, best_fitness, fitness_history = simulated_annealing(SETS, COSTS, universe_size, num_sets)
    
    print(f"Best fitness: {best_fitness}, Elements covered: {valid(best_solution, SETS)}")
    
    # Plotting the fitness history
    plt.plot(fitness_history)
    plt.xlabel('Steps')
    plt.ylabel('Fitness')
    plt.title(f'Fitness trend during optimization (Sets: {num_sets})')
    plt.show()

    return best_solution, best_fitness



#### Initialized all the instances and start the computation

In [ ]:
instances = [
    (100, 10, 0.2),
    (1000, 100, 0.2),
    (10000, 1000, 0.2),
    (100000, 10000, 0.1),
    (100000, 10000, 0.2),
    (100000, 10000, 0.3)
]


for i, (universe_size, num_sets, density) in enumerate(instances):
    print(f"\nInstance {i+1}: Universe = {universe_size}, Sets = {num_sets}, Density = {density}")
    solve_scp_instance(universe_size, num_sets, density)
